# Localization System

## Table of Content 
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#wrangling">Data Wrangling</a></li>
<!-- <li><a href="#eda">Exploratory Data Analysis</a></li> -->
<li><a href="#model"> Building a model </a></li>
</ul>

<a id='intro'></a>
## Introduction 
Localziation system built using machine learning classifiers, data is collected at Cairo University Faculty of Engineering Biomedical Engineering Department. The Data is a collection of WiFi\`s RSSI\`s (dbm) of Wifi Networks available in the department. RSSI measurements represent the relative quality of a received signal on a device. RSSI indicates the power level being received after any possible loss at the antenna and cable level. The higher the RSSI value, the stronger the signal. When measured in negative numbers, the number that is closer to zero usually means better signal. As an example -50 is a pretty good signal, -75 - is fairly reasonable, and -100 is no signal at all. Identifying a patient\`s location in a hospital is useful for many reasons one of which is to identify points of conjestions and try to rearrange the hospital survices, also used to identify how many patients are in the hospital. 

So We are going to dive into our gathered data and invetigate our findings. 

In [69]:
# Basic Importations 
import pandas as pd 
import numpy as np

<a id="#wrangling"></a>
## Data Wrangling 
Let\`s dive into our gathered data and find it\`s secerets. 

In [94]:
# Read our datasets 
df_ta = pd.read_csv('esp_csv_only/csv/Ta.csv')
df_lab = pd.read_csv('esp_csv_only/csv/Lab.csv')
df_ts = pd.read_csv('esp_csv_only/csv/ts.csv')
df_hall_4 = pd.read_csv('esp_csv_only/csv/hall_4.csv')
df_hall_5 = pd.read_csv('esp_csv_only/csv/hall_5.csv')
df_hall_6 = pd.read_csv('esp_csv_only/csv/hall_6.csv')
df_main_hall = pd.read_csv('esp_csv_only/csv/main_hall.csv')
# Show Heads 
df_lab.describe()

,StudBME1,STUDBME2,SBME_STAFF3,SBME_STAFF,CUFE,RehabLab,lab001,BMEStudentLab3,CMP_LAB,CMP_LAB1,CMP_LAB2
count,149.000000,149.000000,149.000000,149.000000,149.000000,149.000000,149.000000,149.0,149.0,149.000000,149.000000
mean,-81.624161,-57.053691,-87.214765,-77.194631,-68.946309,-87.946309,-84.315436,0.0,0.0,-58.570470,-60.677852
std,12.310442,4.658304,2.255815,27.815919,35.480826,2.189611,4.304440,0.0,0.0,30.234948,31.482356
min,-89.000000,-73.000000,-93.000000,-89.000000,-90.000000,-94.000000,-92.000000,0.0,0.0,-85.000000,-87.000000
25%,-85.000000,-60.000000,-89.000000,-88.000000,-88.000000,-89.000000,-87.000000,0.0,0.0,-74.000000,-77.000000
50%,-84.000000,-58.000000,-87.000000,-87.000000,-87.000000,-88.000000,-85.000000,0.0,0.0,-73.000000,-74.000000
75%,-82.000000,-54.000000,-86.000000,-86.000000,-85.000000,-87.000000,-82.000000,0.0,0.0,-71.000000,-71.000000
max,0.000000,-47.000000,-80.000000,0.000000,0.000000,-78.000000,-71.000000,0.0,0.0,0.000000,0.000000


In [95]:
df_ta.describe()

,StudBME1,STUDBME2,SBME_STAFF3,SBME_STAFF,CUFE,RehabLab,lab001,BMEStudentLab3,CMP_LAB,CMP_LAB1,CMP_LAB2
count,151.000000,151.000000,151.000000,151.000000,151.000000,151.000000,151.000000,151.0,151.0,151.000000,151.000000
mean,-78.980132,-69.589404,-71.913907,-84.940397,-83.238411,-68.847682,-86.317881,0.0,0.0,-77.874172,-75.317881
std,10.841568,9.522795,35.969420,10.426397,19.977557,37.952909,10.471785,0.0,0.0,20.161781,25.352021
min,-92.000000,-85.000000,-95.000000,-95.000000,-91.000000,-93.000000,-95.000000,0.0,0.0,-90.000000,-89.000000
25%,-84.000000,-73.000000,-91.000000,-88.000000,-91.000000,-90.000000,-89.000000,0.0,0.0,-86.500000,-86.000000
50%,-80.000000,-70.000000,-91.000000,-86.000000,-87.000000,-90.000000,-88.000000,0.0,0.0,-82.000000,-83.000000
75%,-76.000000,-67.000000,-86.000000,-85.000000,-85.000000,-86.500000,-86.000000,0.0,0.0,-80.000000,-81.000000
max,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000


The first look into the data insights the following:

- BMEStudentLab3 & CMP_LAB are out of range and their values wont benefit our model. 
- There are a lot of zeros in the recorded data.

### The Zeros problem :
The data is recorded in a way that, if one of the specified wifi networks is not found or out of range it records the value for that moment as zero, according to the documentation of the ESP Module the RSSI\`s of a wifi network is a value that ranges from -100 to -50 dB, -100 for bad network connectivity (Signal Strength) and -50 good network connnectivity.
<img src="sources/wifi.jpeg">
So we can treat a zero value as miss recorded value (seems to be an error in the chip) and replace it with any method we want, we will stick with replacing with the mean for now. 

## Imputing miss calculated records 

In [96]:
# Drop the two bad coloumns 
bad_cols = [df_lab.BMEStudentLab3.name, df_lab.CMP_LAB.name]

# dfs classes 1, 2, 3, 4, 5, 6
dfs = [df_ta, df_lab, df_ts, df_hall_4, df_hall_5, df_hall_6, df_main_hall]

for df in dfs:
    df.drop(bad_cols, axis=1, inplace=True)

imp_cols = ['StudBME1', 'STUDBME2', 'SBME_STAFF3', 'SBME_STAFF', 'CUFE',
       'RehabLab', 'lab001', 'CMP_LAB1', 'CMP_LAB2']

# Check 
print([(df.columns.values == imp_cols).all() for df in dfs])

[True, True, True, True, True, True, True]


In [97]:
(df.columns.values == imp_cols).all()

True

In [98]:
# Replace Zeros with the mean
cols = df_lab.columns

for df in dfs:
    df[cols] = df[cols].replace({0: np.nan})

# Check 
print([(df[imp_cols].isnull().sum().values > 0).any() for df in dfs])

[True, True, False, False, False, False, True]


In [99]:
miss = []

for df in dfs:
        missing = [col for col in df if df[col].isnull().sum()>0]
        miss.append(missing)

        
# for i in df_lab.columns:
#     missing_lab = df_lab[i].isnull().sum()
#     missing_ta = df_ta[i].isnull().sum()
    
#     if missing_lab >0 :
#         miss_lab.append(i)
        
#     if missing_ta >0:
#         miss_ta.append(i)
# print(miss_lab)
# print(miss_ta)
miss

[['StudBME1',
  'STUDBME2',
  'SBME_STAFF3',
  'SBME_STAFF',
  'CUFE',
  'RehabLab',
  'lab001',
  'CMP_LAB1',
  'CMP_LAB2'],
 ['StudBME1', 'SBME_STAFF', 'CUFE', 'CMP_LAB1', 'CMP_LAB2'],
 [],
 [],
 [],
 [],
 ['CUFE', 'RehabLab', 'lab001']]

In [100]:
# Replace missing with mean 
for idx, df_cols in enumerate(miss):
    for df_col in df_cols:
        dfs[idx][df_col].fillna(round(dfs[idx][df_col].mean(),1), inplace=True)

In [101]:
# Set Classes for each location
for idx, df in enumerate(dfs):
    df['location'] = idx+1

In [102]:
df_lab.describe()

,StudBME1,STUDBME2,SBME_STAFF3,SBME_STAFF,CUFE,RehabLab,lab001,CMP_LAB1,CMP_LAB2,location
count,149.000000,149.000000,149.000000,149.000000,149.000000,149.000000,149.000000,149.000000,149.000000,149.0
mean,-83.301342,-57.053691,-87.214765,-87.132215,-87.067785,-87.946309,-84.315436,-73.966443,-76.614765,2.0
std,3.704216,4.658304,2.255815,1.044824,1.275087,2.189611,4.304440,2.613417,4.165372,0.0
min,-89.000000,-73.000000,-93.000000,-89.000000,-90.000000,-94.000000,-92.000000,-85.000000,-87.000000,2.0
25%,-85.000000,-60.000000,-89.000000,-88.000000,-88.000000,-89.000000,-87.000000,-74.000000,-77.000000,2.0
50%,-84.000000,-58.000000,-87.000000,-87.100000,-87.000000,-88.000000,-85.000000,-73.000000,-76.000000,2.0
75%,-82.000000,-54.000000,-86.000000,-86.000000,-87.000000,-87.000000,-82.000000,-73.000000,-74.000000,2.0
max,-73.000000,-47.000000,-80.000000,-85.000000,-84.000000,-78.000000,-71.000000,-69.000000,-69.000000,2.0


In [103]:
df_ta.describe()

,StudBME1,STUDBME2,SBME_STAFF3,SBME_STAFF,CUFE,RehabLab,lab001,CMP_LAB1,CMP_LAB2,location
count,151.000000,151.000000,151.000000,151.000000,151.000000,151.000000,151.000000,151.000000,151.000000,151.0
mean,-80.039735,-70.523179,-89.735099,-86.080795,-87.895364,-89.615894,-87.476821,-82.809272,-83.622517,1.0
std,5.766433,5.024718,1.741846,3.349661,2.983764,0.845860,2.996853,4.424076,3.588763,0.0
min,-92.000000,-85.000000,-95.000000,-95.000000,-91.000000,-93.000000,-95.000000,-90.000000,-89.000000,1.0
25%,-84.000000,-73.000000,-91.000000,-88.000000,-91.000000,-90.000000,-89.000000,-86.500000,-86.000000,1.0
50%,-80.000000,-70.500000,-91.000000,-86.000000,-87.900000,-90.000000,-88.000000,-82.800000,-83.600000,1.0
75%,-76.500000,-68.000000,-88.000000,-85.000000,-85.000000,-89.600000,-86.500000,-80.000000,-82.000000,1.0
max,-62.000000,-54.000000,-86.000000,-71.000000,-82.000000,-86.000000,-78.000000,-69.000000,-70.000000,1.0


In [104]:
df_ts.head()

,StudBME1,STUDBME2,SBME_STAFF3,SBME_STAFF,CUFE,RehabLab,lab001,CMP_LAB1,CMP_LAB2,location
0,-81,-55,-55,-88,-73,-50,-86,-62,-63,3
1,-83,-55,-58,-88,-72,-61,-86,-62,-63,3
2,-84,-70,-60,-88,-72,-55,-86,-58,-63,3
3,-84,-72,-58,-88,-71,-47,-86,-58,-63,3
4,-84,-71,-59,-88,-71,-57,-86,-67,-63,3


In [105]:
df_hall_4.head()

,StudBME1,STUDBME2,SBME_STAFF3,SBME_STAFF,CUFE,RehabLab,lab001,CMP_LAB1,CMP_LAB2,location
0,-58,-46,-84,-60,-83,-87,-70,-69,-65,4
1,-58,-51,-84,-69,-83,-94,-79,-65,-65,4
2,-65,-47,-84,-69,-79,-89,-74,-65,-65,4
3,-65,-46,-86,-80,-79,-87,-68,-58,-58,4
4,-65,-47,-81,-66,-72,-85,-72,-60,-53,4


In [106]:
df_hall_5.head()

,StudBME1,STUDBME2,SBME_STAFF3,SBME_STAFF,CUFE,RehabLab,lab001,CMP_LAB1,CMP_LAB2,location
0,-60,-55,-63,-86,-73,-72,-86,-55,-58,5
1,-60,-54,-70,-74,-68,-79,-86,-51,-53,5
2,-60,-54,-76,-74,-76,-72,-86,-53,-53,5
3,-60,-54,-68,-74,-68,-71,-88,-50,-53,5
4,-61,-57,-80,-78,-71,-72,-84,-53,-56,5


In [107]:
df_hall_6.head()

,StudBME1,STUDBME2,SBME_STAFF3,SBME_STAFF,CUFE,RehabLab,lab001,CMP_LAB1,CMP_LAB2,location
0,-74,-59,-84,-68,-73,-81,-83,-67,-67,6
1,-66,-58,-75,-71,-70,-81,-83,-56,-56,6
2,-66,-56,-77,-74,-65,-75,-83,-60,-61,6
3,-66,-55,-78,-74,-63,-80,-83,-60,-52,6
4,-73,-59,-76,-74,-68,-82,-86,-60,-52,6


In [108]:
df = pd.concat(dfs, ignore_index=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1053 entries, 0 to 1052
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   StudBME1     1053 non-null   float64
 1   STUDBME2     1053 non-null   float64
 2   SBME_STAFF3  1053 non-null   float64
 3   SBME_STAFF   1053 non-null   float64
 4   CUFE         1053 non-null   float64
 5   RehabLab     1053 non-null   float64
 6   lab001       1053 non-null   float64
 7   CMP_LAB1     1053 non-null   float64
 8   CMP_LAB2     1053 non-null   float64
 9   location     1053 non-null   int64  
dtypes: float64(9), int64(1)
memory usage: 82.4 KB


In [109]:
df = df.sample(frac=1)

In [110]:
df.head()

,StudBME1,STUDBME2,SBME_STAFF3,SBME_STAFF,CUFE,RehabLab,lab001,CMP_LAB1,CMP_LAB2,location
942,-53.0,-54.0,-84.0,-65.0,-83.0,-88.0,-84.0,-72.0,-75.0,7
415,-83.0,-80.0,-60.0,-88.0,-76.0,-52.0,-92.0,-64.0,-64.0,3
1034,-59.0,-70.0,-87.0,-68.0,-85.0,-87.0,-93.0,-72.0,-85.0,7
1025,-50.0,-71.0,-91.0,-60.0,-85.0,-90.0,-87.0,-72.0,-75.0,7
606,-61.0,-57.0,-80.0,-78.0,-71.0,-72.0,-84.0,-53.0,-56.0,5


<a id='model'></a>
## Building the Model 

In [111]:
# Importations 
from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn import datasets
from sklearn.svm import SVC
from sklearn.metrics import precision_score, recall_score, confusion_matrix, roc_auc_score, classification_report

In [112]:
# Get target and feautre variables 
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

In [129]:
classifiers = {
    'knn': KNeighborsClassifier(5),
    'NB': GaussianNB(),
    'tree': DecisionTreeClassifier(max_depth=10),
    'forest': RandomForestClassifier(n_estimators=10, max_depth=10),
    'SV': SVC(probability=True, gamma=0.001),
    'LR': LogisticRegression(solver='newton-cg', max_iter=400)
}

In [130]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [131]:
# Check Balancing 
pd.Series(y_train).value_counts()

1    130
4    125
3    123
2    119
6    116
7    115
5    114
dtype: int64

In [132]:
X_train

array([[-85. , -73. , -89.7, ..., -87. , -82.8, -83.6],
       [-74. , -67. , -70. , ..., -85. , -58. , -55. ],
       [-61. , -52. , -74. , ..., -85. , -64. , -63. ],
       ...,
       [-81. , -60. , -91. , ..., -91. , -80. , -86. ],
       [-65. , -49. , -75. , ..., -84. , -67. , -66. ],
       [-57. , -48. , -92. , ..., -76. , -66. , -69. ]])

In [133]:
# Metrics 
metrics = ['accuracy', 'f1_weighted', 'roc_auc_ovr_weighted', 'precision_weighted', 'recall_weighted']

# Cross Validate the models 
scores = {}
for name, clf in classifiers.items():
    print(f'Training {name} model .. ')
    res = cross_validate(clf, X_train, y_train, cv=10, scoring=metrics, return_estimator=True)
#     print(res)
    
    score = {"test_f1_weighted": np.mean(res['test_f1_weighted']), 
             'test_roc_auc_ovr_weighted': np.mean(res['test_roc_auc_ovr_weighted']), 
             'test_precision_weighted': np.mean(res['test_precision_weighted']),
             'test_recall_weighted': np.mean(res['test_recall_weighted']),
             'test_accuracy': np.mean(res['test_accuracy']),
             'estimator': res['estimator'][0]
            }
    
    print("test_f1_weighted", score['test_f1_weighted'])
    print('test_roc_auc_ovr', score['test_roc_auc_ovr_weighted'])
    print('test_precicison', score['test_precision_weighted'])
    print('test_recall', score['test_recall_weighted'])
    print('accuracy', score['test_accuracy'])
    print('\n')
    
    # Add each model`s scores to scores
    scores[name] = score

Training knn model .. 
test_f1_weighted 0.9242605028935984
test_roc_auc_ovr 0.9879528074789203
test_precicison 0.9311762476883894
test_recall 0.9240896358543417
accuracy 0.9240896358543417


Training NB model .. 
test_f1_weighted 0.9247372473663116
test_roc_auc_ovr 0.9950229841759844
test_precicison 0.9310516696966277
test_recall 0.9252661064425769
accuracy 0.9252661064425769


Training tree model .. 
test_f1_weighted 0.9031556931363717
test_roc_auc_ovr 0.9535155866106597
test_precicison 0.9095557841823597
test_recall 0.9038515406162464
accuracy 0.9038515406162464


Training forest model .. 
test_f1_weighted 0.9315342892296533
test_roc_auc_ovr 0.9946583525310997
test_precicison 0.9399909474279221
test_recall 0.9311624649859944
accuracy 0.9311624649859944


Training SV model .. 
test_f1_weighted 0.9444652231468618
test_roc_auc_ovr 0.9964478568669305
test_precicison 0.9497913920813081
test_recall 0.9442717086834733
accuracy 0.9442717086834733


Training LR model .. 
test_f1_weighted 0.93

### Desicion 

According to our application we decided to choose our model according to the F1 Score

In [147]:
# Find the best estimator according to one metric
choosen = 'test_' + metrics[4]
best_metric = [(name, res[choosen]) for name, res in scores.items()]
best_estimator = max(best_metric, key=lambda x: x[1])
print(f"The best estimator is {best_estimator[0]} with Score {best_estimator[1]} ({choosen})")

The best estimator is SV with Score 0.9442717086834733 (test_recall_weighted)


In [148]:
# Fetch Estimator Object
# choosen_estimator = scores[best_estimator[0]]['estimator']
choosen_estimator = classifiers[best_estimator[0]]
choosen_estimator

SVC(gamma=0.001, probability=True)

In [149]:
# Train the choosen model
choosen_estimator.fit(X_train, y_train)

SVC(gamma=0.001, probability=True)

In [150]:
# Predict Using the choosen estimator 
res = choosen_estimator.predict(X_test)

#### Reporting the results 

In [151]:
# Print the confusion matrix
confusion_matrix(y_test, res)

array([[21,  0,  0,  0,  0,  0,  0],
       [ 1, 29,  0,  0,  0,  0,  0],
       [ 0,  0, 28,  0,  0,  0,  0],
       [ 0,  0,  0, 20,  3,  0,  3],
       [ 0,  0,  0,  6, 27,  4,  0],
       [ 0,  0,  0,  0,  2, 32,  0],
       [ 1,  0,  0,  0,  0,  0, 34]])

In [152]:
# Show The classification Report
print(classification_report(y_test, res))

              precision    recall  f1-score   support

           1       0.91      1.00      0.95        21
           2       1.00      0.97      0.98        30
           3       1.00      1.00      1.00        28
           4       0.77      0.77      0.77        26
           5       0.84      0.73      0.78        37
           6       0.89      0.94      0.91        34
           7       0.92      0.97      0.94        35

    accuracy                           0.91       211
   macro avg       0.90      0.91      0.91       211
weighted avg       0.90      0.91      0.90       211



In [153]:
# Save the model 
from joblib import dump

In [154]:
dump(choosen_estimator, 'child/model.joblib')

['child/model.joblib']

#### Tunning 

In [156]:
hp = {
    'C': [0.1, 0.01, 0.001],
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid', 'precomputed'],
    'poly': [3, 4, 5],
    'gamma': ['scale', 'auto'],
    'max_itr': [1000, 2000, -1],
    
}

In [157]:
gridres = GridSearchCV(choosen_estimator, hp, scoring=metrics, n_jobs=3)

In [158]:
gridres.fit(X, y)

ValueError: For multi-metric scoring, the parameter refit must be set to a scorer key or a callable to refit an estimator with the best parameter setting on the whole data and make the best_* attributes available for that metric. If this is not needed, refit should be set to False explicitly. True was passed.